In [139]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

data = pd.read_csv("task-3-dataset.csv")
corpus = data["отзывы"]
fin = data["разметка"].to_list()
fin = [1 if i=="+" else 0 for i in fin]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

sequences = tokenizer.texts_to_sequences(corpus)
vocab_size = len(tokenizer.word_index) + 1

In [118]:
context_target_pairs = []
window_size = 2

for sequence in sequences:
    for i, target_word in enumerate(sequence):
        for j in range(max(0, i - window_size), min(len(sequence), i + window_size + 1)):
            if j != i:
                context_target_pairs.append((target_word, sequence[j]))


In [132]:
import numpy as np

X, y = zip(*context_target_pairs)
X = np.array(X)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [120]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense


embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1))
model.add(Flatten())
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')



In [121]:
num_epochs = 10

model.fit(X, y, epochs=num_epochs)

Epoch 1/10
318/318 [==============================] - 3s 8ms/step - loss: 7.0767
Epoch 2/10
318/318 [==============================] - 3s 10ms/step - loss: 6.7617
Epoch 3/10
318/318 [==============================] - 2s 6ms/step - loss: 6.3471
Epoch 4/10
318/318 [==============================] - 2s 6ms/step - loss: 6.0221
Epoch 5/10
318/318 [==============================] - 2s 6ms/step - loss: 5.7149
Epoch 6/10
318/318 [==============================] - 2s 6ms/step - loss: 5.4021
Epoch 7/10
318/318 [==============================] - 2s 6ms/step - loss: 5.0915
Epoch 8/10
318/318 [==============================] - 4s 13ms/step - loss: 4.7949
Epoch 9/10
318/318 [==============================] - 2s 7ms/step - loss: 4.5223
Epoch 10/10
318/318 [==============================] - 2s 6ms/step - loss: 4.2785


In [122]:
word_embeddings = model.layers[0].get_weights()[0]

target_word = 'телефон'
word_index = tokenizer.word_index.get(target_word)
if word_index is not None:
    word_embedding_vector = word_embeddings[word_index]
    print(f"Вектор для слова '{target_word}': {word_embedding_vector}")
else:
    print(f"'{target_word}' не найдено среди всех слов.")

Вектор для слова 'телефон': [-0.35741407  0.1339267  -0.4532818  -0.15932302 -0.04418449 -0.28421944
  0.47274002  0.16375524 -0.01170467 -0.3986193   0.27419224 -0.3159022
  0.53526103  0.4247141   0.10579167 -0.33013615 -0.22953078 -0.46438876
 -0.38823265 -0.4344085  -0.5206725  -0.20210354  0.28918138  0.310456
 -0.1979416   0.33816576 -0.41567242 -0.11800007  0.5352778   0.6013583
 -0.4347485  -0.2045411   0.14448817 -0.21696346 -0.38845488 -0.04558199
 -0.28810078  0.34633395 -0.06831435 -0.31511036 -0.20751542  0.5536184
 -0.09212263 -0.45571676  0.4745746   0.24388722  0.17282708 -0.1442755
 -0.07850103  0.22250332  0.5367127   0.4424646   0.00283869  0.04340453
 -0.31778008  0.2366      0.05797578  0.04110621  0.06463521  0.42512706
 -0.3353674  -0.29857287  0.122361    0.18611804  0.6407472   0.2775411
 -0.41764265 -0.07232455  0.13853529 -0.54698354  0.4037067   0.38135242
 -0.31432185 -0.05389244  0.01324827  0.67270625 -0.23603547  0.45949396
  0.1657486  -0.29210037 -0.30

In [123]:
import cirq
def Anzatz(qubits, thetax, thetay):
    n = len(qubits)
    yield [cirq.Rx(rads=thetax[j])(qubits[j]) for j in range(n)]
    yield [cirq.Ry(rads=thetay[j])(qubits[j]) for j in range(n)]
    yield [cirq.CNOT(qubits[j], qubits[j + 1]) for j in range(n - 1)]
    yield [cirq.M(i) for i in qubits]

In [135]:
def get_ising_operator(qubits):
    def kron(l):
        if len(l) != 1:
            res = np.kron(l[-2], l[-1])
            l = l[:-2]
            l.append(res)
            return kron(l)
        else:
            return l[0]
    for i in range(len(qubits)):
       op = cirq.Z(qubits[i])
    return np.abs(op - fin)

In [141]:
import tensorflow_quantum as tfq
import sympy
from tqdm import tqdm

n = 4
epochs = 10
qubits = cirq.LineQubit.range(n)
op = get_ising_operator(qubits)
thetax = sympy.symbols(f"x0:{n}")
thetay = sympy.symbols(f"y0:{n}")
circuit = cirq.Circuit()
circuit.append(Anzatz(qubits, thetax, thetay))
lr = 0.1

model = tfq.layers.SampledExpectation()

# Вектор обучаемых параметров
teta = tf.Variable(2*np.pi*np.random.random((1, 2*n)))

log_writer = tf.summary.create_file_writer("train")

for epoch in tqdm(range(epochs)):
    with tf.GradientTape() as gt:
        out = model(
            circuit,
            symbol_names=thetax + thetay,
            symbol_values=teta,
            operators=op,
            repetitions=500
        )

    grad = gt.gradient(out, teta)
    thetas.assign_sub(lr * grad)

    with log_writer.as_default():
        tf.summary.scalar("Eigen Val", out[0, 0], step=epoch)
        tf.summary.histogram("Gradients", grad, step=epoch)

TypeError: unsupported operand type(s) for -: 'SingleQubitPauliStringGateOperation' and 'list'